In [ ]:
import os
import pandas as pd
import numpy as np
import librosa # For audio loading and spectrogram
import logging
from tqdm import tqdm
import random
import jax # Import jax if you want to use jax.random for augmentation
import jax.numpy as jnp # Or use numpy for augmentation if you prefer

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Cấu hình chung ---
DATASET_ROOT = 'dataset'
SAMPLE_RATE = 16000 # Phải khớp với SAMPLE_RATE trong prepare_real_audio.py
N_MELS = 128        # Số lượng Mel bands
N_FFT = 2048        # Kích thước cửa sổ FFT
HOP_LENGTH = 512    # Bước nhảy giữa các cửa sổ FFT
MAX_AUDIO_DURATION = 10 # Giây (phải khớp với MAX_LEN_SEC trong prepare_real_audio.py)
TARGET_SPEC_WIDTH = 256 # Chiều rộng (thời gian) mục tiêu của spectrogram (e.g., 256 frames * HOP_LENGTH/SR ~ 8 giây)
                        # Điều chỉnh kích thước này cho phù hợp với model ViT/CNN của bạn.
MIN_CLIP_DURATION = 5 # Giây (phải khớp với MIN_LEN_SEC trong prepare_real_audio.py)

# Constants for Spectrogram calculation
WIN_LENGTH = N_FFT

class AudioDeepfakeDatasetJAX:
    def __init__(self, metadata_path, dataset_root, sample_rate, n_mels, n_fft, hop_length, 
                 target_spec_width, augment=None, label_mapping={'real': 0, 'fake': 1}):
        self.metadata_df = pd.read_csv(metadata_path)
        self.dataset_root = dataset_root
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.target_spec_width = target_spec_width
        self.augment = augment # Data augmentation function (e.g., SpecAugment)
        self.label_mapping = label_mapping

        # Tính toán số lượng frames tương ứng với MAX_AUDIO_DURATION
        # frames = (duration_sec * sample_rate) / hop_length
        self.max_frames = int((MAX_AUDIO_DURATION * self.sample_rate) / self.hop_length)

        # Lọc bỏ các dòng có đường dẫn audio bị NaN (có thể do lỗi xử lý trước đó)
        self.metadata_df = self.metadata_df.dropna(subset=['path'])
        
        # Tiền xử lý tất cả dữ liệu thành spectrograms và lưu vào bộ nhớ (hoặc disk nếu dataset quá lớn)
        # Đối với dataset kích thước vừa phải, lưu vào RAM là nhanh nhất.
        self.processed_data = []
        logging.info(f"Bắt đầu tiền xử lý và tải dữ liệu từ '{metadata_path}' vào bộ nhớ...")
        for idx in tqdm(range(len(self.metadata_df)), desc="Processing audio files"):
            spec, label, fake_level, path = self._process_single_item(idx)
            if spec is not None:
                self.processed_data.append((spec, label, fake_level, path))
            else:
                logging.warning(f"Bỏ qua mục bị lỗi tại index {idx} trong {metadata_path}")
        logging.info(f"Đã tải và tiền xử lý {len(self.processed_data)} mẫu.")

    def __len__(self):
        return len(self.processed_data)

    def _process_single_item(self, idx):
        row = self.metadata_df.iloc[idx]
        audio_path_relative = row['path']
        label_str = row['label']
        fake_level = row.get('fake_level', 0) # Mặc định là 0 cho real

        full_audio_path = os.path.join(self.dataset_root, audio_path_relative)

        try:
            # Tải audio
            waveform, sr = librosa.load(full_audio_path, sr=None) # sr=None để giữ sample rate gốc

            # Resample nếu cần
            if sr != self.sample_rate:
                waveform = librosa.resample(y=waveform, orig_sr=sr, target_sr=self.sample_rate)

            # Convert waveform to Mel-spectrogram
            # librosa.feature.melspectrogram trả về (n_mels, n_frames)
            mel_spec = librosa.feature.melspectrogram(
                y=waveform,
                sr=self.sample_rate,
                n_fft=self.n_fft,
                hop_length=self.hop_length,
                n_mels=self.n_mels,
                win_length=WIN_LENGTH
            )
            
            # Chuyển sang log-Mel spectrogram (định dạng log-power)
            # Add a small constant to avoid log(0)
            mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 
            
            # Chuẩn hóa về [0, 1] hoặc [-1, 1]
            # Đây là một bước quan trọng. Có thể chuẩn hóa trên toàn bộ dataset hoặc từng spectrogram.
            # Với mô hình CNN/ViT, thường cần input chuẩn hóa.
            # Normalizing to [-1, 1] for example:
            mel_spec = (mel_spec - mel_spec.min()) / (mel_spec.max() - mel_spec.min()) * 2 - 1


            # Pad hoặc cắt spectrogram đến kích thước cố định TARGET_SPEC_WIDTH
            current_width = mel_spec.shape[1]
            if current_width < self.target_spec_width:
                # Pad với 0
                pad_amount = self.target_spec_width - current_width
                mel_spec = np.pad(mel_spec, ((0, 0), (0, pad_amount)), mode='constant', constant_values=0)
            elif current_width > self.target_spec_width:
                # Random crop
                start_idx = random.randint(0, current_width - self.target_spec_width)
                mel_spec = mel_spec[:, start_idx : start_idx + self.target_spec_width]

            # Thêm kênh màu (channels-first) cho CNN/ViT, shape: (1, N_MELS, TARGET_SPEC_WIDTH)
            mel_spec = np.expand_dims(mel_spec, axis=0)
            
            label = np.array(self.label_mapping[label_str], dtype=np.int32)
            
            return mel_spec.astype(np.float32), label, fake_level, row['path']

        except Exception as e:
            logging.error(f"Lỗi khi tải hoặc xử lý audio {full_audio_path}: {e}")
            return None, None, None, None
            
    def __getitem__(self, idx):
        # Trả về dữ liệu đã tiền xử lý từ bộ nhớ
        return self.processed_data[idx]

# --- Data Augmentation (JAX-compatible) ---
# Chúng ta sẽ viết các hàm augmentation có thể JIT biên dịch
def spec_augment(spec, key, freq_mask_param=30, time_mask_param=100, num_freq_masks=1, num_time_masks=1):
    """
    Apply SpecAugment to a spectrogram.
    Args:
        spec (jnp.ndarray): Spectrogram of shape (channels, N_MELS, TARGET_SPEC_WIDTH).
        key (jax.random.PRNGKey): JAX random key for reproducibility.
        freq_mask_param (int): Max width of frequency mask.
        time_mask_param (int): Max width of time mask.
        num_freq_masks (int): Number of frequency masks.
        num_time_masks (int): Number of time masks.
    Returns:
        jnp.ndarray: Augmented spectrogram.
    """
    
    # Ensure spec is JAX array
    spec = jnp.asarray(spec)

    _, n_mels, n_frames = spec.shape
    
    for _ in range(num_freq_masks):
        key, subkey = jax.random.split(key)
        f = jax.random.randint(subkey, (), 0, freq_mask_param)
        key, subkey = jax.random.split(key)
        f0 = jax.random.randint(subkey, (), 0, n_mels - f)
        spec = spec.at[:, f0:f0+f, :].set(0.0) # Set to 0 (or mean)
        
    for _ in range(num_time_masks):
        key, subkey = jax.random.split(key)
        t = jax.random.randint(subkey, (), 0, time_mask_param)
        key, subkey = jax.random.split(key)
        t0 = jax.random.randint(subkey, (), 0, n_frames - t)
        spec = spec.at[:, :, t0:t0+t].set(0.0) # Set to 0 (or mean)
        
    return spec

# --- Batch Generator ---
def data_generator(dataset, batch_size, rng_key=None, shuffle=True, repeat=False):
    """
    A generator that yields batches of data (NumPy arrays).
    """
    data_indices = list(range(len(dataset)))
    
    while True:
        if shuffle:
            random.shuffle(data_indices)
        
        for i in range(0, len(data_indices), batch_size):
            batch_indices = data_indices[i:i + batch_size]
            
            batch_specs = []
            batch_labels = []
            batch_fake_levels = []
            batch_paths = []

            for idx in batch_indices:
                spec, label, fake_level, path = dataset[idx]
                batch_specs.append(spec)
                batch_labels.append(label)
                batch_fake_levels.append(fake_level)
                batch_paths.append(path)
            
            # Stack into NumPy arrays
            yield np.stack(batch_specs), np.stack(batch_labels), np.array(batch_fake_levels), batch_paths
        
        if not repeat:
            break

# --- Hàm để khởi tạo Dataset và Generator ---
def get_data_generator(set_type, batch_size, rng_key, shuffle=True, repeat=False, num_workers=0, include_fake_levels=None):
    """
    Initializes the dataset and returns a batch generator.
    num_workers is a placeholder for potential future multiprocessing.
    """
    metadata_path = os.path.join(DATASET_ROOT, set_type, f'combined_metadata_{set_type}.csv')
    
    df_raw = pd.read_csv(metadata_path)
    
    # Lọc dữ liệu theo fake_level nếu yêu cầu
    if include_fake_levels is not None:
        if not isinstance(include_fake_levels, list):
            include_fake_levels = [include_fake_levels]
        # Tạo mask để lọc
        mask = df_raw['fake_level'].isin(include_fake_levels)
        # Đặc biệt xử lý trường hợp fake_level 0 (real)
        if 0 in include_fake_levels:
            mask = mask | (df_raw['fake_level'] == 0)
        df_filtered = df_raw[mask].copy()
        logging.info(f"Đã lọc dataset '{set_type}' để chỉ bao gồm fake_level: {include_fake_levels}. Số lượng mẫu: {len(df_filtered)}")
    else:
        df_filtered = df_raw.copy()

    # Tạo một bản sao của DataFrame đã lọc cho dataset
    # Cần một dataset riêng cho mỗi lần gọi get_data_generator nếu muốn lọc khác nhau
    dataset = AudioDeepfakeDatasetJAX(
        metadata_path=metadata_path, # Vẫn truyền metadata path gốc để tải tất cả
        dataset_root=DATASET_ROOT,
        sample_rate=SAMPLE_RATE,
        n_mels=N_MELS,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        target_spec_width=TARGET_SPEC_WIDTH,
        augment=spec_augment if set_type == 'train' else None # Chỉ áp dụng augmentation cho tập train
    )
    # Cập nhật metadata_df của dataset để nó chỉ chứa dữ liệu đã lọc
    # Note: processed_data đã được tạo từ metadata_df ban đầu, nên cần lọc lại processed_data
    original_processed_data = dataset.processed_data
    dataset.processed_data = []
    
    # Lọc processed_data dựa trên df_filtered
    filtered_paths = set(df_filtered['path'].tolist())
    for spec, label, fake_level, path in original_processed_data:
        if path in filtered_paths:
            dataset.processed_data.append((spec, label, fake_level, path))

    logging.info(f"Số lượng mẫu sau khi lọc cho '{set_type}': {len(dataset.processed_data)}")

    return data_generator(dataset, batch_size, rng_key=rng_key, shuffle=shuffle, repeat=repeat)


# --- Ví dụ sử dụng ---
if __name__ == "__main__":
    # Khởi tạo JAX RNG key
    key = jax.random.PRNGKey(0)

    # Test Data Generator
    logging.info("Kiểm tra Data Generator cho tập train...")
    train_gen = get_data_generator('train', batch_size=4, rng_key=key, shuffle=True, 
                                   include_fake_levels=[0, 1, 3, 4]) # Bao gồm real và các mức fake
    
    # Lấy một vài batch
    for i, (specs, labels, fake_levels, paths) in enumerate(train_gen):
        print(f"\nBatch {i+1}:")
        print(f"  Spectrograms shape: {specs.shape}") # Expected: (batch_size, 1, N_MELS, TARGET_SPEC_WIDTH)
        print(f"  Labels: {labels}")
        print(f"  Fake Levels: {fake_levels}")
        print(f"  Paths (first 2): {paths[:2]}")
        
        # Test augmentation if it's applied
        if i == 0 and 'train' in train_gen.__self__.dataset.metadata_df['path'].iloc[0] and train_gen.__self__.dataset.augment:
             # Apply augmentation to a sample spec (need a JAX key for this)
            sub_key = jax.random.split(key)[0] # Just take one subkey for demonstration
            augmented_spec = spec_augment(specs[0], sub_key)
            print(f"  Augmented spec shape: {augmented_spec.shape}")
            # You can add visualization here if needed (e.g., matplotlib)
            
        if i >= 1: # Chỉ lấy 2 batch để kiểm tra
            break

    logging.info("\nKiểm tra Data Generator cho tập val (không shuffle, không augment)...")
    val_gen = get_data_generator('val', batch_size=4, rng_key=key, shuffle=False)
    for i, (specs, labels, fake_levels, paths) in enumerate(val_gen):
        print(f"\nBatch {i+1}:")
        print(f"  Spectrograms shape: {specs.shape}")
        print(f"  Labels: {labels}")
        print(f"  Fake Levels: {fake_levels}")
        if i >= 0:
            break

    logging.info("\nKiểm tra Data Generator cho tập test (không shuffle, không augment)...")
    test_gen = get_data_generator('test', batch_size=4, rng_key=key, shuffle=False)
    for i, (specs, labels, fake_levels, paths) in enumerate(test_gen):
        print(f"\nBatch {i+1}:")
        print(f"  Spectrograms shape: {specs.shape}")
        print(f"  Labels: {labels}")
        print(f"  Fake Levels: {fake_levels}")
        if i >= 0:
            break